In [1]:
from fastapi import FastAPI, Request, status
from fastapi.responses import Response, StreamingResponse
import nest_asyncio
nest_asyncio.apply()
import uvicorn
from pydantic import BaseModel
from dotenv import load_dotenv
load_dotenv()
import openai
import os
import sys
from pathlib import Path
from openai import OpenAI
import ast
import json
from fastapi.middleware.cors import CORSMiddleware

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not len(OPENAI_API_KEY):
    print("Set OPENAI_API_KEY")


In [3]:
class Speech(BaseModel):
    excerpt:str
    previous_suggestion:str
    topic: str

app = FastAPI()

origins = [
    "http://localhost:4321",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials = True,
    allow_methods=["*"],
    allow_headers=["*"]
)




In [4]:
def generate_speech(excerpt, previous_suggestion):
    prompt = [{"role":"system","content":'''Your job is to continue my speech in 1 sentence. Your output should be a json with only one key, which is next_sentence. Refer to your previous suggestion. 
               If user is following your previous suggestion. The value of next_sentence should be the part of the previous suggestion that is not yet included in the excerpt plus a new sentence. Else,
               the value of next_sentence should just be a new sentence.'''},
              {"role":"user","content":f"Previous suggestion: {previous_suggestion}"},
              {"role":"user","content":f"Speech so far: {excerpt}"}]
    return prompt

In [ ]:
@app.post("/generate_speech",status_code=status.HTTP_202_ACCEPTED)
def continue_speech(speech:Speech):
    try:
        client = OpenAI()
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo-0125",
            response_format={"type":"json_object"},
            temperature=0.7,
            messages=generate_speech(excerpt=speech.excerpt,previous_suggestion=speech.previous_suggestion)
        )
        return response.choices[0].message.content
    except Exception as e:
        print("ERROR!")
        return {"Error":e}

# uvicorn.run(app, host="192.168.0.23",port=8080)
uvicorn.run(app, host="192.168.68.142",port=8080)

In [5]:
def generate_speech_async(excerpt, previous_suggestion,topic):
    prompt = [{"role":"system","content":'''Your job is to continue my speech in 1 sentence. Do not say anything else. 
    Your response should be string. I read your suggestions, but I can choose not to follow them. If I am not yet done
    reading your previous suggestion, output your previous suggestion. Else, output a new sentence to continue my speech.'''},
        {"role":"user","content":f"Topic: {topic}"},
        {"role":"user","content":f"Speech so far: {excerpt}"},
        {"role":"user","content":f"Previous suggestion: {previous_suggestion}"}
    ]
    return prompt

async def standard_stream_generator(response, **kwargs):
    msg = ""
    for chunk in response:
        try:
            chunk_response = chunk.choices[0].delta.content
            msg += chunk_response
            print(msg)
            yield msg.encode()
        except:
            yield ""
            continue

@app.post("/generate_speech_async",status_code=status.HTTP_202_ACCEPTED)
async def continue_speech_async(speech:Speech):
    messages = generate_speech_async(excerpt = speech.excerpt, previous_suggestion = speech.previous_suggestion, topic = speech.topic)
    print(messages)
    try:
        client = OpenAI()
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            temperature = 0.7,
            messages = messages,
            stream = True
        )
        return StreamingResponse(standard_stream_generator(response,excerpt = speech.excerpt, previous_suggestion = speech.previous_suggestion,
                                    topic = speech.topic),
                                    media_type='text/event-stream')

    except Exception as e:
        print(e)
        return ""
uvicorn.run(app,host="192.168.68.142",port=8080)

INFO:     Started server process [6656]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://192.168.68.142:8080 (Press CTRL+C to quit)


INFO:     192.168.68.142:61759 - "POST /generate_speech_async HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\LXPH\anaconda3\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\LXPH\anaconda3\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\LXPH\anaconda3\lib\site-packages\fastapi\applications.py", line 289, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\LXPH\anaconda3\lib\site-packages\starlette\applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\LXPH\anaconda3\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "c:\Users\LXPH\anaconda3\lib\site-packages\starlette\middleware\errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "c:\Users\LXPH\anaconda3\